#**Chat with Multiple PDF's Llama 2 + Pinecone + LangChain**

#**Step: 01: Install All the Required Packages**

In [1]:
# !pip install langchain
# !pip install pinecone-client
# !pip install sentence_transformers
# !pip install pdf2image
# !pip install pypdf
# !pip install xformers
# !pip install bitsandbytes accelerate transformers

#**Step 2: Import All the Required Libraries**

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
import os
import sys
import torch

c:\Users\benha\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA

#**Step 3: Load the PDF Files**

In [4]:
!mkdir pdfs

A subdirectory or file pdfs already exists.


#**Step 04: Extract the text from the PDF's**

In [5]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [6]:
print(data)

[Document(page_content='Thermodynamics  ', metadata={'source': 'pdfs\\Chapter_1.pdf', 'page': 0}), Document(page_content='Information:  \nContact:  \n\uf0a7Email: maria.fahed@inervenants.efrei.fr  \n       Contact me trough email or @teams for any question.  \nNumber of classes:  \n \n \n \nEvaluation:  \n\uf0a7Midterm exam: 30%  \n\uf0a7Final exam: 50%  \n\uf0a7Tutorials: 20%  Modules  Tutorials  \n7 7 \nChapters:  \n•Fundamentals of Thermodynamics   \n•First Law of Thermodynamics    \n•Second  Law of Thermodynamics     \n•Thermal machines   ', metadata={'source': 'pdfs\\Chapter_1.pdf', 'page': 1}), Document(page_content='Fundamentals of Thermodynamics   Thermodynamics  \nClass nb 1  ', metadata={'source': 'pdfs\\Chapter_1.pdf', 'page': 2}), Document(page_content='Course outline  \nI.Introduction  \nII.States of matter  \nIII.Scales of study  \nIV.Thermodynamic system  \nV.Ideal and Van Der  Waals gases  \nVI.Thermoelastic  coefficients  ', metadata={'source': 'pdfs\\Chapter_1.pdf', '

#**Step 05: Split the Extracted Data into Text Chunks**

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=384, chunk_overlap=20)

In [8]:
docs = text_splitter.split_documents(data)

In [9]:
print(len(docs))

276


In [10]:
docs[3]

Document(page_content='Fundamentals of Thermodynamics   Thermodynamics  \nClass nb 1', metadata={'source': 'pdfs\\Chapter_1.pdf', 'page': 2})

In [11]:
docs[4]

Document(page_content='Course outline  \nI.Introduction  \nII.States of matter  \nIII.Scales of study  \nIV.Thermodynamic system  \nV.Ideal and Van Der  Waals gases  \nVI.Thermoelastic  coefficients', metadata={'source': 'pdfs\\Chapter_1.pdf', 'page': 3})

#**Step 06: Downlaod the Embeddings From the Hugging Face**

In [12]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [13]:
query_result = embeddings.embed_query("Hello World")

In [14]:
print("Length", len(query_result))

Length 384


#**Step 07: Initializing the Pinecone**

In [15]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', 'e625eff7-7e12-48a1-aab4-2d53d3670ce7')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [16]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "chatbotpdfs" # put in the name of your pinecone index here


#**Step 08: Create Embeddings for each of the Text Chunk**

In [17]:
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

#**Step 09: Similarity Search**

In [18]:
query = "how many students must we have in each team"

In [19]:
docs = docsearch.similarity_search(query, k=4)

In [20]:
docs

[Document(page_content='•Any request for a 3 member team must be approved by your professor beforehand.\n1'),
 Document(page_content='•Any request for a 3 member team must be approved by your professor beforehand.\n1'),
 Document(page_content='•Any request for a 3 member team must be approved by your professor beforehand.\n1'),
 Document(page_content='•Any request for a 3 member team must be approved by your professor beforehand.\n1')]

#**Step 10: Calling the API**

In [21]:
# !pip install llama-cpp-python

In [22]:
from llama_cpp import Llama
# load the model from 2 pytorch_model-00001-of-00002.bin files pytorch_model-00002-of-00002.bin
LLM = Llama("ggml-model-q4_k.gguf")

# prompt = "Q: What is the capital of France? A:"

# output = LLM(prompt)
# output


AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [23]:
SYSTEM_PROMPT = """Use the following pieces of context to answer the question at the end. and if the contenxt is relevant to the question, please answer the question while incorporating the context into your answer. """

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"

SYSTEM_PROMPT = B_SYS + SYSTEM_PROMPT + E_SYS

instruction = """
{context}

Question: {question}
Answer: 
"""

template = SYSTEM_PROMPT + instruction

template

'<>\nUse the following pieces of context to answer the question at the end. and if the contenxt is relevant to the question, please answer the question while incorporating the context into your answer. \n<>\n\n\n{context}\n\nQuestion: {question}\nAnswer: \n'

In [24]:
# # create a loop to ask questions and use the docsearch to et context
# question = "how does recomender system work?"
# # append the 4 context paragraphs
# context = docsearch.similarity_search(question, k=2)
# for c in context:
#     print(c.page_content)
#     print("--------------")
# # context = "-".join([c.page_content for c in context])
# # concatenate the context elements in one string
# context = " \n ".join([c.page_content for c in context])
# context = context.replace("\n", " ")
# prompt_template = PromptTemplate(template=template, input_variables=["context", "question"])
# prompt = prompt_template.format(context=context, question=question)
# print("Prompt:", prompt)
# output = LLM(prompt, max_tokens=500, 
#     stop=["Q:"],
#     echo=True)


In [25]:
# output

In [26]:
# output["usage"]

In [27]:
# print only the completion tokens and not the prompt
# print("Answer : " + output["choices"][0]["text"][len(prompt):])

In [28]:
import discord
import regex as re
import asyncio
import nest_asyncio
nest_asyncio.apply()
DISCORD_TOKEN = "MTE3NDMzNTgyNzIxNjcxNTg1OA.GkiifU.PRObWCZ5Ibh1eYR2scBBVdBCogDweJ5gcLu-dQ"


class MyClient(discord.Client):
    async def on_ready(self):
        print('Logged in as ' + str(self.user.name))
        print('The chatbot Id :: ' + str(self.user.id))

    async def on_message(self, message):
        if message.author == self.user:  # Check if the message is from the bot itself
            return
        print(message.content)
        # if message contains ? then it is a question
        if "?" in message.content:
            question = message.content
            # append the 4 context paragraphs
            context = docsearch.similarity_search(question, k=4)
            for c in context:
                print(c.page_content)
                print("--------------")
            # context = "-".join([c.page_content for c in context])
            # concatenate the context elements in one string
            context = " \n ".join([c.page_content for c in context])
            context = context.replace("\n", " ")
            prompt_template = PromptTemplate(template=template, input_variables=["context", "question"])
            prompt = prompt_template.format(context=context, question=question)
            print("Prompt:", prompt)
            output = LLM(prompt, max_tokens=500,
                         stop=["Q:"],
                         echo=True)
            # print only the completion tokens and not the prompt
            print("Answer : " + output["choices"][0]["text"][len(prompt):])
            await message.channel.send(output["choices"][0]["text"][len(prompt):])


async def main():
    intents = discord.Intents.default()
    intents.message_content = True  # Enable message content intent

    client = MyClient(intents=intents)
    await client.start(DISCORD_TOKEN)


if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(main())


intents = discord.Intents.default()
intents.message_content = True  # Enable message content intent

client = MyClient(intents=intents)
client.run(DISCORD_TOKEN)



Logged in as chatbot
The chatbot Id :: 1174335827216715858
Quels sont les freins qui existent aujourd'hui pour faire de l'hydrogène, l'énergie du futur pour les transports ?
Course outline  
I.Introduction  
II.States of matter  
III.Scales of study  
IV.Thermodynamic system  
V.Ideal and Van Der  Waals gases  
VI.Thermoelastic  coefficients
--------------
Course outline  
I.Introduction  
II.States of matter  
III.Scales of study  
IV.Thermodynamic system  
V.Ideal and Van Der  Waals gases  
VI.Thermoelastic  coefficients
--------------
Course outline  
I.Introduction  
II.States of matter  
III.Scales of study  
IV.Thermodynamic system  
V.Ideal and Van Der  Waals gases  
VI.Thermoelastic  coefficients
--------------
Course outline  
I.Introduction  
II.States of matter  
III.Scales of study  
IV.Thermodynamic system  
V.Ideal and Van Der  Waals gases  
VI.Thermoelastic  coefficients
--------------
Prompt: <>
Use the following pieces of context to answer the question at the end. and 